In [1]:
import pandas as pd
import numpy as np
import sqlparse
from sentence_transformers import SentenceTransformer, util

desc_col = {}

description_df = pd.read_excel('src/New_query_Description.xlsx',header=1)
description_df = description_df[['Column','Description']].dropna()
x = 0
for i, row in description_df.iterrows():
    desc_col[x] = row['Column']
    x += 1

In [49]:
def get_col(sql):
    col = []
    ignore = ['team_tds.tds_intern.pointx_fbs_txn_rpt_dly','sum','avg','mean','count']
    for token in sql.tokens:
        # print(type(token),token)
        if str(token).lower() in ignore:
            continue
        if isinstance(token, sqlparse.sql.Identifier):
            col.append(str(token))
        elif isinstance(token, sqlparse.sql.IdentifierList) or isinstance(token, sqlparse.sql.Where) or isinstance(token, sqlparse.sql.Having) or isinstance(token, sqlparse.sql.Comparison) or isinstance(token, sqlparse.sql.Function) or isinstance(token, sqlparse.sql.Parenthesis):
            col += get_col(token)
    return col

In [50]:
descriptions = description_df['Description'].tolist()

In [51]:
pointx_cols = pd.read_csv('src/pointx_fbs_rpt_dly.csv').columns.to_list()
assert len(pointx_cols) == len(desc_col), "Length of columns in Pointx table and Descriptions are not equal"

In [52]:
compare_df = pd.read_csv('src/compare_result.csv')
questions = compare_df['Question'].to_list()
sql_queries = compare_df['SQL'].to_list()

In [53]:
model = SentenceTransformer('all-MiniLM-L6-v2')

def get_col_max_score(question,description_df=description_df,model=model):
    q_emb = model.encode(question)
    scores = [float(util.cos_sim(q_emb, model.encode(des))) for des in description_df['Description'].to_list()]
    print("COLUMN:",description_df.iloc[np.argmax(scores)]['Column'])
    print("DESC:",description_df.iloc[np.argmax(scores)]['Description'])
    print("SCORE:",np.max(scores))
    print()
    
    return int(np.argmax(scores))

In [58]:
for i, q in enumerate(questions[:10]):
    print(q)
    sql_parse = sqlparse.parse(sql_queries[i])[0]
    # print(sql_parse)
    print('LABEL COLUMNS:',list(set(get_col(sql_parse))))
    get_col_max_score(q)

How many daily active users each day?
LABEL COLUMNS: ['engagement_time_msec', 'event_date', 'user_pseudo_id', 'ga_session_id', 'event_date ASC']
COLUMN: deal_type
DESC: Deal of the day type
SCORE: 0.39395755529403687

How many monthly active users each month?
LABEL COLUMNS: ['engagement_time_msec', 'event_month', 'user_pseudo_id', 'ga_session_id', 'event_month ASC']
COLUMN: privacy_info_ads_storage
DESC: Whether ad targeting is enabled for a user.
SCORE: 0.3766669034957886

What is the average number of daily active users last 7 days?
LABEL COLUMNS: ['engagement_time_msec', 'GETDATE', 'event_date', 'user_pseudo_id', 'DATEADD', 'ga_session_id', 'cnt', 'COUNT(DISTINCT user_pseudo_id) as cnt']
COLUMN: user_ltv_revenue
DESC: The Lifetime Value (revenue) of the user. This field is not populated in intraday tables.
SCORE: 0.42309463024139404

What is the mean number of daily active users from last 30 days?
LABEL COLUMNS: ['engagement_time_msec', 'GETDATE', 'event_date', 'user_pseudo_id', 'DA

In [57]:
for token in sqlparse.parse(s)[0].tokens:
    print(type(token),token)
    if isinstance(token,sqlparse.sql.IdentifierList):
        print(token.tokens)

<class 'sqlparse.sql.Token'> SELECT
<class 'sqlparse.sql.Token'>  
<class 'sqlparse.sql.Function'> AVG(cnt)
<class 'sqlparse.sql.Token'>  
<class 'sqlparse.sql.Token'> FROM
<class 'sqlparse.sql.Token'> 

<class 'sqlparse.sql.Token'>  
<class 'sqlparse.sql.Token'>  
<class 'sqlparse.sql.Parenthesis'> (SELECT event_date, COUNT(DISTINCT user_pseudo_id) as cnt
  FROM (
      SELECT event_date, ga_session_id, user_pseudo_id
      FROM team_tds.tds_intern.pointx_fbs_txn_rpt_dly
      GROUP BY event_date, ga_session_id, user_pseudo_id
      HAVING SUM(engagement_time_msec) > 10*1000
  )
  GROUP BY event_date
  ORDER BY event_date ASC)
<class 'sqlparse.sql.Token'> 

<class 'sqlparse.sql.Where'> WHERE DAYOFWEEK(event_date) IN (1, 7)


In [56]:
s = """SELECT AVG(cnt) FROM
  (SELECT event_date, COUNT(DISTINCT user_pseudo_id) as cnt
  FROM (
      SELECT event_date, ga_session_id, user_pseudo_id
      FROM team_tds.tds_intern.pointx_fbs_txn_rpt_dly
      GROUP BY event_date, ga_session_id, user_pseudo_id
      HAVING SUM(engagement_time_msec) > 10*1000
  )
  GROUP BY event_date
  ORDER BY event_date ASC)
WHERE DAYOFWEEK(event_date) IN (1, 7)"""